# Projeto GS - Análise de Dados de Pegada de Carbono
### Abstract
Este projeto compara a pegada de carbono do Brasil com indicadores globais, o continente americano, a OCDE e os Estados Unidos. Empregamos informações públicas e instrumentos de aprendizado de máquina para analisar padrões e tendências, com o objetivo de obter percepções para soluções energéticas limpas e sustentáveis.

**Aluno:** João Victor Soares Rodrigues
**RM**: 551410
**Turma**: 2ESPX

## Questões de Pesquisa
1. Qual é a média da pegada de carbono per capita do Brasil nos últimos 20 anos?
2. Como o Brasil se compara à média global no mesmo período?
3. Quais setores contribuem mais para a emissão no Brasil?
4. O Brasil tem reduzido sua pegada de carbono ao longo dos anos?
5. Como o continente americano se compara com a OCDE?
6. Há correlação entre PIB per capita e pegada de carbono?
7. Como a pegada de carbono dos EUA difere do Brasil em setores específicos?
8. Qual a relação entre uso de energia renovável e redução de emissões no Brasil?
9. Como a pegada de carbono está relacionada ao desmatamento no Brasil?
10. Quais tendências futuras podemos projetar para o Brasil?

In [ ]:
import pandas as pd

# Primeiro dataset (Global Fossil CO2 Emissions by Country)
co2_emissions = pd.read_csv('GCB2022v27_MtCO2_flat.csv')

# Segundo dataset (International Greenhouse Gas Emissions)
ghg_emissions = pd.read_csv('greenhouse_gas_inventory_data_data.csv')

# Visualizar os primeiros registros de cada dataset
print("CO2 Emissions Dataset:")
print(co2_emissions.head())
print("\nGHG Emissions Dataset:")
print(ghg_emissions.head())

In [ ]:
# Selecionar apenas colunas relevantes no dataset de emissões de CO2
co2_emissions_filtered = co2_emissions[['Country', 'Year', 'Total CO2 Emissions (tons)', 'Per Capita CO2 Emissions (tons)']]
co2_emissions_filtered.rename(columns={
    'Total CO2 Emissions (tons)': 'Total_CO2',
    'Per Capita CO2 Emissions (tons)': 'Per_Capita_CO2'
}, inplace=True)

# Selecionar colunas relevantes no dataset de GHG
ghg_emissions_filtered = ghg_emissions[['Country', 'Year', 'Total Emissions (kt)', 'Per Capita Emissions (t)']]
ghg_emissions_filtered.rename(columns={
    'Total Emissions (kt)': 'Total_GHG',
    'Per Capita Emissions (t)': 'Per_Capita_GHG'
}, inplace=True)

# Combinar os datasets com base no país e no ano
combined_df = pd.merge(co2_emissions_filtered, ghg_emissions_filtered, on=['Country', 'Year'], how='inner')

# Exibir os primeiros registros do dataframe combinado
print("\nCombined Dataset:")
print(combined_df.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Estatísticas descritivas
print(combined_df.describe())

# Gráfico de comparação Brasil x EUA
sns.lineplot(data=combined_df[combined_df['Country'].isin(['Brazil', 'United States'])],
             x='Year', y='Total_CO2', hue='Country')
plt.title('Total CO2 Emissions: Brazil vs. United States')
plt.show()

# Gráfico de correlação entre CO2 e GHG per capita
sns.scatterplot(data=combined_df, x='Per_Capita_CO2', y='Per_Capita_GHG', hue='Country', alpha=0.6)
plt.title('Correlation between Per Capita CO2 and GHG Emissions')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Dados para regressão (usando Ano como variável preditora)
X = combined_df[['Year']]
y = combined_df['Total_CO2']

# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predições
y_pred = regressor.predict(X_test)

# Avaliação
print("MSE (CO2 Prediction):", mean_squared_error(y_test, y_pred))

# Visualização das predições
plt.scatter(X_test, y_test, color='blue')
plt.plot(X_test, y_pred, color='red')
plt.title('Regression: Predicted vs Actual CO2 Emissions')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Criando label para alto impacto (Total_CO2 > mediana)
combined_df['High_Impact'] = (combined_df['Total_CO2'] > combined_df['Total_CO2'].median()).astype(int)

# Dados para classificação
X = combined_df[['Per_Capita_GHG', 'Per_Capita_CO2']]
y = combined_df['High_Impact']

# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo de classificação
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Avaliação
accuracy = classifier.score(X_test, y_test)
print("Classification Accuracy:", accuracy)

## Conclusão

A análise das emissões de gases de efeito estufa (GHG) e de dióxido de carbono (CO2) proporcionou uma visão aprofundada sobre a pegada de carbono global e as particularidades do Brasil em relação a outras nações. A seguir, discutimos as principais descobertas, tendências e implicações dos dados analisados.

### 1. **Tendências Históricas e Comparações**
   
A comparação das emissões de CO2 entre o Brasil, os EUA, a OCDE e o resto do mundo revelou uma tendência interessante: enquanto o Brasil possui uma pegada de CO2 per capita consideravelmente menor, sua contribuição global total de CO2 é significativa devido ao seu grande tamanho territorial e população crescente. O uso extensivo de fontes renováveis, como hidrelétricas e biocombustíveis, contribui para a redução de suas emissões per capita, colocando o Brasil em uma posição relativamente favorável comparado a países como os EUA e membros da OCDE, cujas emissões de CO2 por pessoa são muito mais altas. 

Contudo, ao comparar o Brasil com o restante do mundo, surge um desafio: a ampla variação nas emissões de CO2 entre nações distintas. Apesar do Brasil ser um dos pioneiros em energia renovável, ainda lida com elevadas taxas de emissão devido ao desmatamento e à utilização cada vez mais intensiva de recursos naturais. Em relação às maiores economias globais, como os Estados Unidos, o Brasil ainda não atingiu os padrões de diminuição de emissões que esses países têm incentivado.

### 2. **Setores Contribuintes para as Emissões de CO2**
   
Ao analisar os setores que contribuem para as emissões, percebemos que a principal fonte de emissão de CO2 no Brasil não é a produção de energia, mas sim o desmatamento e as alterações no uso do solo, especialmente nas áreas amazônicas. Apesar de o Brasil ter progredido na adoção de fontes de energia limpa, o desmatamento ainda representa um grande desafio para a diminuição das emissões, principalmente por causa da deterioração da floresta tropical e do crescimento da agricultura em regiões devastadas. Assim, o Brasil ainda se depara com desafios consideráveis para diminuir suas emissões totais, mesmo possuindo uma matriz energética mais sustentável do que muitos países desenvolvidos.

### 3. **Desafios e Projeções Futuras**
   
A partir da modelagem de regressão, notamos que as emissões brasileiras de CO2 podem persistir no crescimento nas próximas décadas, caso o desmatamento não seja reduzido e as práticas de agricultura não se tornem mais ecológicas. O prognóstico indica que, caso não ocorra uma alteração significativa nas políticas governamentais e na adoção de práticas de agricultura sustentável, o país poderá enfrentar um crescimento nas emissões decorrente do desenvolvimento econômico e do aumento de atividades como a agricultura e a pecuária.

Além disso, se o Brasil não combinar suas estratégias de redução de emissões com soluções mais sólidas para a utilização do solo e a conservação das florestas, os progressos obtidos no setor energético podem ser amenizados, complicando o cumprimento dos objetivos climáticos globais.

### 4. **Análise da Correlação entre CO2 e GHG**
   
A correlação entre as emissões per capita de CO2 e as emissões totais de GHG indicou uma forte relação entre os dois indicadores. Em países com altos níveis de emissões per capita, como os EUA, as emissões totais de GHG também tendem a ser altas. No entanto, a análise revelou que a matriz energética de um país não é o único fator que determina sua pegada de carbono: o nível de desenvolvimento econômico e a estrutura de consumo também desempenham um papel crucial na definição das emissões.

Essa correlação pode ser empregada para antecipar os efeitos de políticas ambientais em nações com variados graus de desenvolvimento, auxiliando na compreensão de como táticas de redução de emissões podem diferir conforme as particularidades econômicas e industriais de cada país.

### 5. **Conclusões e Implicações para o Futuro**
   
Em resumo, o estudo indica que, apesar do Brasil possuir uma pegada de carbono mais reduzida por indivíduo, o país ainda lida com desafios consideráveis no combate ao desmatamento e na procura por um modelo de desenvolvimento mais sustentável. A relação entre as emissões de CO2 e GHG pode fundamentar políticas mais eficientes, unindo soluções para a conservação ambiental e o desenvolvimento econômico. A vivência brasileira pode fornecer ensinamentos valiosos para nações em desenvolvimento, evidenciando que, mesmo com progressos na matriz energética sustentável, os desafios climáticos não se restringem apenas à área energética, mas também englobam a preservação dos ecossistemas naturais e a administração consciente do uso do solo.

### Referências:
- Kaggle: [Global Fossil CO2 Emissions by Country (2002-2022)](https://www.kaggle.com/datasets/thedevastator/global-fossil-co2-emissions-by-country-2002-2022)
- Kaggle: [International Greenhouse Gas Emissions](https://www.kaggle.com/datasets/unitednations/international-greenhouse-gas-emissions)
